# Introduction

Ce TP continue le TP précédent. Nous allons reprendre d'ailleurs les mêmes données et commencer la mise en oeuvre d'un modèle de Markov pour la prédiction des étiquettes: 
* une observation est une phrase, représentée comme une séquence de variables aléatoires, une par mot de la phrase
* à cette observation est associée une séquence de variables aléatoires représentant les étiquettes, une par mot de la phrase également

On suppose que la séquence d'observation (une phrase) est générée par un modèle de Markov caché. Les variables cachées sont donc les étiquettes à inférer. Nous allons commencer par écrire une classe python pour représenter le HMM. Cette classe évoluera au fil des TPs. 

Pour cela le code de départ suivant est donné. Afin d'initialiser un HMM, nous devons connaitre : 
- l'ensemble des états (ou *state_list*), dans notre cas l'ensemble des étiquettes grammaticales;
- l'ensemble des observations (ou *observation_list*), dans notre cas l'ensemble des mots connus; tous les autres mots seront remplacés par l'élément spécial *UNK* qui fait partie de l'ensemble des observations. 

Enfin, en interne il est plus facile d'indexer les mots et et les états par des entiers. Ainsi à chaque éléments de respectivement l'ensemble des états et l'ensemble des observations, est associé un indice. Cela nous permet de tout traiter en "matricielle". 

In [127]:
import nltk
from numpy import array, ones, zeros
import sys

# Some words in test could be unseen during training, or out of the vocabulary (OOV) even during the training. 
# To manage OOVs, all words out the vocabulary are mapped on a special token: UNK defined as follows: 
UNK = "<unk>" 
UNKid = 0 

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None):
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print("HMM created with: ")
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            print(str(self.N)+" states")
            print(str(self.M)+" observations")
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of tags
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            omega_Y_keys = [key for key in self.omega_Y.keys()]
            omega_X_keys = [key for key in self.omega_X.keys()]
            for i in range(self.N):
                self.Y_index[omega_Y_keys[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[omega_X_keys[i]] = i
                
        def get_X_index(self, word):
            if word in self.X_index:
                return word
            else:
                return UNK
        
        def compute_init_state_proba(self):
            vals = np.array([val for val in self.omega_Y.values()])
            sum_vals = sum(vals)
            self.initial_state_proba =  vals / sum_vals
            
        def compute_observation_probas(self, data):            
            for phr in data:
                for word in phr:
                    x = self.X_index[self.get_X_index(word[0])]
                    y = self.Y_index[word[1]]
                    self.observation_proba[x][y] += 1
            self.observation_proba /= np.sum(self.observation_proba, axis=1)[:, np.newaxis]
             
        def compute_transition_probas(self, data):            
            for phr in data:
                for i in range(1, len(phr)):
                    yminus1 = self.Y_index[phr[i - 1][1]]
                    y = self.Y_index[phr[i][1]]
                    self.transition_proba[yminus1][y] += 1
            self.transition_proba /= np.sum(self.transition_proba, axis=1)[:, np.newaxis]
            
        def init_parameters(self, train_set):
            self.compute_init_state_proba()
            self.compute_observation_probas(train_set)
            self.compute_transition_probas(train_set)
            
        def forward(self, obs):
            alpha = np.zeros((len(obs), len(self.Y_index)))
            alpha[0] = self.initial_state_proba * self.observation_proba[self.X_index[self.get_X_index(obs[0][0])]]
            for i in range(1, len(alpha)):
                alpha[i] = self.observation_proba[self.X_index[self.get_X_index(obs[i][0])]] *\
                np.sum(self.transition_proba * alpha[i - 1])
            return alpha
        
        def backward(self, obs):
            beta = np.zeros((len(obs), len(self.Y_index)))
            beta[-1] = ones(len(self.Y_index))
            for i in range(len(obs) - 2, -1, -1):
                beta[i] = np.sum(beta[i + 1] * self.observation_proba[self.X_index[self.get_X_index(obs[i + 1][0])]]\
                                 * self.transition_proba)
            return beta
        
        def decode(self, alpha, beta):
            prob = alpha * beta
            preds = prob.argmax(axis=1)
            keys = [key for key in self.omega_Y.keys()]
            return [keys[pred_ind] for pred_ind in preds]
        
        def viterbi(self, obs):
            mu = np.zeros((len(obs), len(self.Y_index)))
            mu_max = np.zeros(len(obs))
            mu[0] = self.initial_state_proba * self.observation_proba[self.X_index[self.get_X_index(obs[0][0])]]
            index = [np.argmax(mu[0])]
            mu_max[0] = max(mu[0])
            for i in range(1, len(obs)):
                obs_prob = self.observation_proba[self.X_index[self.get_X_index(obs[i][0])]]
                trans_prob = self.transition_proba[self.Y_index[obs[i][1]]]
                mu[i] = obs_prob * trans_prob * mu_max[i - 1]
                index.append(np.argmax(mu[i]))
                mu_max[i] = max(mu[i])
            keys = [key for key in self.omega_Y.keys()]
            return [keys[ind] for ind in index]
            
        def score_eval(self, test):
            error = 0
            elements = 0
            for sentence in test:
                truth = [tag for (_, tag) in sentence]
                alpha = self.forward(sentence)
                beta = self.backward(sentence)
                preds = self.decode(alpha, beta)
                elements += len(preds)
                for pred, y in zip(truth, preds):
                    if pred != y:
                        error += 1
            return error / elements
        
        def score_viterbi(self, test):
            error = 0
            elements = 0
            for sentence in test:
                truth = [tag for (_, tag) in sentence]
                preds = self.viterbi(sentence)
                elements += len(preds)
                for pred, y in zip(truth, preds):
                    if pred != y:
                        error += 1
            return error / elements

# Interface avec les données et apprentissage supervisé

Ainsi pour initialiser un HMM, nous allons devoir lire les données (chose faîte lors du TP précédent): 
* écrire une fonction permettant d'initialiser le HMM à partir des données d'apprentissage
* écrire une fonction *apprentissage_supervisé* qui permet d'estimer les paramètres 

Dans un premier temps, nous limiterons, comme lors du TP précédent, le vocabulaire aux mots apparaissant 10 fois ou plus. Les autres mots sont tous remplacés par la même forme *unk*

Pour cela, le plan de travail peut être envisagé ainsi: 
* Lire les données puis générer un corpus de **train** (80%) puis de **test** (10%)
* écrire une fonction qui créer à partir des données d'apprentissage (**train**), tous les comptes nécessaires pour l'estimation supervisée des paramètres du HMM
* écrire 3 fonctions qui estiment les paramètres à partir des comptes, une fonction par distribution: observation, transition, état initial. 
* écrire une fonction qui reprend le tout et qui estime tous les paramètres du HMM


# Exercice : Algorithme de Viterbi

La question qui se pose est comment calculer la meilleure séquence d'étiquettes pour une phrase donnée connaissant les paramètres du HMM. Par meilleure, on entend la séquence d'étiquettes (ou d'états) la plus probable connaissant la séquence d'obervation. 

Proposer et implémenter un algorithme répondant à cette question. Pour vous aider à démarrer, cet algorithme s'appelle Viterbi et regardez cette vidéo https://www.youtube.com/watch?v=RwwfUICZLsA, pour comprendre comment il opère. 

# TODO pour la prochaine fois

* Finir la partie interface (qui comprend l'apprentissage supervisé)
* Regarder la vidéo et implémenter l'algorithme de Viterbi




## Lecture & séparation des données

In [3]:
import pickle
import numpy as np

In [5]:
data = pickle.load(open("brown.save.p", "rb" ))

In [6]:
np.random.shuffle(data)
end_train = int(0.8 * len(data))
end_valid = int(0.9 * len(data))
train = data[:end_train]
valid = data[end_train:end_valid]
test = data[end_valid:]

## Création du vocabulaire & du HMM

In [7]:
def clean_voc(dict_word, count_min):
    new_dict = dict()
    new_dict['<unk>'] = 0
    for key, val in dict_word.items():
        if val < count_min: 
            new_dict['<unk>'] += val
        else:
            new_dict[key] = val
    return new_dict

def distrib_x_y_data(data):
    set_tag = []
    set_mot = []
    dict_tag = dict()
    dict_mot = dict()
    for phrase in data:
        for mot in phrase:
            if not(mot[1] in set_tag):
                set_tag.append(mot[1])
                dict_tag[mot[1]]=0
            dict_tag[mot[1]]+=1
            if not(mot[0] in set_mot):
                set_mot.append(mot[0])
                dict_mot[mot[0]]=0
            dict_mot[mot[0]]+=1
    return clean_voc(dict_mot, 10), dict_tag

In [9]:
X_train, y_train = distrib_x_y_data(train)

In [128]:
hmm_train = HMM(y_train, X_train)

HMM created with: 
12 states
7603 observations


In [129]:
hmm_train.init_parameters(train)

In [130]:
print(hmm_train.omega_Y)

{'ADJ': 67119, 'NOUN': 220121, 'ADP': 115720, 'DET': 109684, 'VERB': 146264, 'ADV': 44892, 'PRT': 23813, '.': 117951, 'PRON': 39491, 'NUM': 11815, 'CONJ': 30527, 'X': 1138}


In [131]:
sent = train[15]
print(sent)

[('Yin', 'NOUN'), ('and', 'CONJ'), ('Yang', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('``', '.'), ('Lo', 'NOUN'), ('Shu', 'NOUN'), ("''", '.'), ('square', 'NOUN')]


In [132]:
vit = hmm_train.viterbi(sent)
print(vit)

['NOUN', 'CONJ', 'NOUN', 'ADP', 'DET', '.', 'NOUN', 'NOUN', '.', 'NOUN']


In [133]:
error_valid = hmm_train.score_eval(valid)
viterbi_error_valid = hmm_train.score_viterbi(valid)

In [134]:
print("Error = {:.2%}".format(error_valid))
print("Error = {:.2%}".format(viterbi_error_valid))

Error = 8.54%
Error = 10.46%


In [135]:
error_test = hmm_train.score_eval(test)
viterbi_error_test = hmm_train.score_viterbi(test)

In [136]:
print("Error = {:.2%}".format(error_test))
print("Error = {:.2%}".format(viterbi_error_test))

Error = 8.64%
Error = 10.58%
